In [17]:
import numpy as np
import matplotlib.pyplot as plt
from fixed_params import *
from fixed_params.general import *
from fixed_params.propulsion import *
from fixed_params.avionics import *
from fixed_params.flight_track import *
from fixed_params.wing import *
from material_properties import areal_mass
from skopt import gp_minimize
from skopt.space import Real, Integer
from scipy.optimize import brentq

## Mission Scoring Functions

In [18]:
def round_inches(x: float) -> float:
    """
    Round a measurement in inches down to the nearest 0.00, 0.25, 0.50, or 0.75.

    Example:
        10.37 -> 10.25
        5.81  -> 5.75
        7.99  -> 7.75
    """
    # Whole inch part
    whole = int(x)

    # Fractional part
    frac = x - whole

    # Define breakpoints
    targets = [0.00, 0.25, 0.50, 0.75]

    # Find the largest target <= frac
    rounded_frac = max([t for t in targets if t <= frac], default=0.00)

    return whole + rounded_frac

def mission_2(num_passengers, num_cargo, m2_laps, battery_capacity):

    income = (num_passengers * (6 + 2 * m2_laps)) + (num_cargo * (10 + 8 * m2_laps))
    EF = battery_capacity / 100
    cost = m2_laps * (10 + (num_passengers*0.5) + (num_cargo*2)) * EF
    net_income = income - cost
    m2 = 1 + (net_income / 900)
    return m2

def mission_3(banner_length, number_of_laps, wing_span):
    """
    :param banner_length: length in inches
    :param number_of_laps: number of laps
    :param wing_span: span in metres
    :return: mission 3 score with best score of 410
    """
    wing_span_inches = wing_span * 39.3701
    pre = round(wing_span_inches)/12 * 0.05 + 0.75
    RAC = pre if pre >= 0.9 else 0.9
    rounded_banner = round_inches(banner_length)
    m3 = (rounded_banner * number_of_laps / RAC)
    m3_best = 1100
    return 2 + (m3 / m3_best)

### --- Utility Functions (Externalized and Parameterized) ---

In [19]:
def velocity(CL, n, W, S):
    """
    Calculates flight velocity.

    Args:
        CL (float): Lift coefficient.
        n (float): Load factor.
        W (float): Weight (Newtons).
        S (float): Wing area (m^2).

    Returns:
        float: Velocity (m/s).
    """
    return np.sqrt((2 * n * W) / (rho * S * CL))


def power_required(CL, W, CD0, k, S, CD_banner=0):
    """
    Calculates required power for straight and level flight (n=1).

    Args:
        CL (float): Lift coefficient.
        W (float): Weight (Newtons).
        CD0 (float): Zero-lift drag coefficient.
        k (float): Induced drag constant.
        S (float): Wing area (m^2).
        CD_banner (float): Additional banner drag coefficient (default 0).

    Returns:
        float: Required Power (Watts).
    """
    CD = CD0 + CD_banner + k * CL ** 2
    v = velocity(CL, 1, W, S)
    return (CD * W / CL) * v


def find_CL_cruise(W, P_effective, CD0, k, S, CL_min, CL_max, CD_banner=0):
    """
    Finds the lift coefficient for cruise flight where P_effective = P_required.

    Args:
        W (float): Weight (Newtons).
        P_effective (float): Effective motor power (Watts).
        CD0 (float): Zero-lift drag coefficient.
        k (float): Induced drag constant.
        S (float): Wing area (m^2).
        CL_min (float): Minimum searchable lift coefficient.
        CL_max (float): Maximum searchable lift coefficient.
        CD_banner (float): Additional banner drag coefficient (default 0).

    Returns:
        float: Lift coefficient (CL) solution.

    Raises:
        ValueError: If brentq cannot find a root in the specified range.
    """
    # Define the specific power function needed for the search, closing over parameters
    def current_power_required(CL):
        return power_required(CL, W, CD0, k, S, CD_banner)

    CL_grid = np.linspace(CL_min, CL_max, 2000)
    f_vals = P_effective - np.array([current_power_required(CL) for CL in CL_grid])

    # Find bracket where sign changes
    sign_change_indices = np.where(np.sign(f_vals[:-1]) * np.sign(f_vals[1:]) < 0)[0]
    
    if len(sign_change_indices) == 0:
        # If no sign change, return the CL that minimizes the absolute difference
        idx_best = np.argmin(np.abs(f_vals))
        # Note: If min abs difference is still far from zero, the solution is approximate
        return CL_grid[idx_best]

    # Use first bracket found for brentq
    a = CL_grid[sign_change_indices[0]]
    b = CL_grid[sign_change_indices[0] + 1]
    
    try:
        # Pass a lambda that calls the closed-over function
        CL_solution = brentq(lambda CL: P_effective - current_power_required(CL), a, b)
        return CL_solution
    except ValueError as error:
        raise ValueError(f"CL_cruise search error: {error}. Could not find a root between {a:.2f} and {b:.2f}.")


def find_turning_conditions(CL, W, CD0, k, P_effective, S):
    """
    Calculates turning performance (n, V, angle, radius) for a given CL and W.

    Args:
        CL (float): Lift coefficient used for the turn.
        W (float): Weight (Newtons).
        CD0 (float): Zero-lift drag coefficient.
        k (float): Induced drag constant.
        P_effective (float): Effective motor power (Watts).
        S (float): Wing area (m^2).

    Returns:
        tuple: (n_max, v, angle_rad, radius)
    """
    CD = CD0 + k * CL ** 2
    
    # Calculate the power required for straight flight at this CL (A)
    A = (CD * W / CL) * np.sqrt(2.0 * W / (rho * S * CL))
    
    # Calculate the maximum load factor (n_max) possible with P_effective
    n_max = (P_effective / A) ** (2.0 / 3.0) 
    
    # Calculate velocity at n_max
    v = velocity(CL, n_max, W, S)
    
    # Calculate bank angle (rad) and turn radius
    angle = np.arccos(1 / n_max) 
    radius = v ** 2 / (g * np.tan(angle))
    
    return n_max, v, angle, radius


def flight_time_one_lap(radius, v_straight, v_turn):
    """
    Calculates time for one lap of the track.

    Args:
        radius (float): Turn radius (m).
        v_straight (float): Cruise velocity (m/s).
        v_turn (float): Turning velocity (m/s).
        straight_distance (float): Length of the straight segment (m).

    Returns:
        float: Time for one lap (seconds).
    """
    # Assuming turning segment is 4*pi*r total distance (2 turns of 2*pi*r each)
    TURNING_DISTANCE_RATIO = 2 * (2 * np.pi) 
    
    # The original logic included a multiplier of 1.5
    total_time = (straight_distance / v_straight + TURNING_DISTANCE_RATIO * radius / v_turn) * 1.5
    return total_time

## Combined Mission Objective Function

In [20]:
def objective_total(x):
    """
    Calculates the combined (negative) score for Mission 2 and Mission 3.

    Args:
        x (tuple/array): Decision variables (WS, pucks, PC_ratio, banner_length, 
                         banner_AR, m2_fly_time, m3_fly_time).
    """
    # Unpack decision variables
    WS, pucks, PC_ratio, banner_length, banner_AR, m2_fly_time, m3_fly_time = x
    
    # Constants
    TIME_TO = 30  # secs
    TIME_LANDING = 30  # secs
    STRUCTURE_RATIO = 1.5
    USABLE_ENERGY = 0.8
    
    # Derived Payload Calculations
    m_duck = general.duck
    m_puck = general.puck
    ducks = pucks * PC_ratio
    
    m2_payload = pucks * m_puck + ducks * m_duck
    
    # Banner calculations
    banner_length_m = banner_length * 0.0254  # inches to meters
    banner_width = banner_length / banner_AR
    banner_width_m = banner_width * 0.0254
    area_banner = banner_length_m * banner_width_m
    m3_payload = 1.2 * area_banner * areal_mass.lightweight_ripstop
    
    payload = max(m2_payload, m3_payload)
    
    # Total Mass (m_total) and Power Calculations
    m = (1 + STRUCTURE_RATIO) * payload
    m_struct = payload * STRUCTURE_RATIO
    power = general.power_ratio * m
    P_effective = power * motor_eff

    # Battery and Motor Mass (based on Mission 2 flight time as the limiting case)
    # The battery is sized for the maximum required flight time across both missions.
    # Since m2_fly_time and m3_fly_time are decision variables, we'll size for m2_time_flight.
    # NOTE: You should ensure the sizing covers the maximum of (m2_time_flight, m3_time_flight).
    m2_total_time = m2_fly_time + TIME_LANDING + TIME_TO
    m3_total_time = m3_fly_time + TIME_LANDING + TIME_TO
    max_total_time = max(m2_total_time, m3_total_time)
    
    m_motor = power / propulsion.power_density
    m_battery = power * (max_total_time / 3600 / USABLE_ENERGY / avionics.energy_density)
    
    # Mission 2 battery capacity check (original code used m2_time_flight, preserving this for now)
    battery_cap_m2 = power * (m2_total_time / 3600 / USABLE_ENERGY)
    if battery_cap_m2 > 100:
        return 1e6 # Large penalty for capacity over 100 Wh
    
    # Mission 3 battery capacity check (original code used m2_time_flight, preserving this for now)
    battery_cap_m3 = power * (m3_total_time / 3600 / USABLE_ENERGY)
    if battery_cap_m3 > 100:
        return 1e6 # Large penalty for capacity over 100 Wh

    # Wing Geometry
    S = m / wing.wing_loading
    c = S / WS
    AR = WS ** 2 / S

    # Aerodynamic Constants
    CD0 = 0.13 + 0.01 * np.sqrt(ducks)
    k = 1 / (np.pi * AR * e)

    # ------------------------------------------------
    # --- MISSION 2 CALCULATIONS ---
    # ----------------------------------------------------
    
    m_2 = m_struct + m2_payload
    W_2 = m_2 * g

    
    try:
        # Turning performance
        n_turning_2, v_turning_2, bank_angle_rad2, turn_radius_2 = find_turning_conditions(CL_turn, W_2, CD0, k, P_effective, S)
        
        # Cruise performance
        CL_cruise_2 = find_CL_cruise(W_2, P_effective, CD0, k, S, CL_min, CL_max)
        V_cruise_2 = velocity(CL_cruise_2, 1, W_2, S)

        
    except ValueError as error:
        # Penalize if cruise/turning conditions cannot be met
        print(f"M2 Flight Error: {error}")
        return 1e6

    # Laps calculation
    one_lap_time_2 = flight_time_one_lap(turn_radius_2, V_cruise_2, v_turning_2)
    number_of_laps_2 = m2_fly_time // one_lap_time_2
    
    if number_of_laps_2 == 0:
        return 1e6
    
    # Scoring
    try:
        m2_score = mission_2(ducks, pucks, number_of_laps_2, battery_cap_m2)
        if not np.isfinite(m2_score):
            return 1e6

    except Exception as error:
        print(f"M2 Scoring Error: {error}")
        return 1e6
    
    # ----------------------------------------------------
    # --- MISSION 3 CALCULATIONS ---
    # ----------------------------------------------------
    
    m_3 = m_struct + m3_payload
    W_3 = m_3 * g

    # Power required function for M3 (with banner drag)
    CD_banner = 0.5 * np.power(banner_AR, -0.5)
    
    try:
        # Turning performance
        n_turning_3, v_turning_3, bank_angle_rad3, turn_radius_3 = find_turning_conditions(CL_turn, W_3, CD0, k, P_effective, S)
        
        # Cruise performance
        CL_cruise_3 = find_CL_cruise(W_3, P_effective, CD0, k, S, CL_min, CL_max, CD_banner=CD_banner)
        V_cruise_3 = velocity(CL_cruise_3, 1, W_3, S)

        
    except ValueError as error:
        # Penalize if cruise/turning conditions cannot be met
        print(f"M3 Flight Error: {error}")
        return 1e6

    # Laps calculation
    one_lap_time_3 = flight_time_one_lap(turn_radius_3, V_cruise_3, v_turning_3)
    number_of_laps_3 = m3_fly_time // one_lap_time_3
    
    if number_of_laps_3 == 0:
        return 1e6
    
    # Scoring
    try:
        m3_score = mission_3(banner_length, number_of_laps_3, WS)
        if not np.isfinite(m3_score):
            return 1e6
        
    except Exception as error:
        print(f"M3 Scoring Error: {error}")
        return 1e6
    print()
    print("############ General Plane Dimensions ###############")
    print(f"WS: {WS:.2f}")
    print(f"Chord: {c:.2f}")
    print(f"Aspect Ratio: {AR:.2f}")
    print(f"Motor Power: {power:.1f}") # Use 1 decimal for power, maybe
    print(f"CDO: {CD0:.4f}") # Use 4 decimals for CD0, as it's a small coefficient
    print()
    print("############ MISSION INPUTS ###############")
    # Integers don't need formatting, but floats will be formatted
    print(f"Mission 2: Ducks = {ducks}, Pucks = {pucks}, payload = {m2_payload:.2f} kg, Battery Cap = {battery_cap_m2:.0f} Wh")
    # The banner length is already in inches, but formatting the float
    print(f"Mission 3: Banner Length (Inches)= {banner_length:.1f}, Banner AR = {banner_AR:.1f}, payload = {m3_payload:.2f} kg, Battery Cap = {battery_cap_m3:.0f} Wh")
    print()
    print("############ MISSION 2 ##############")
    print(f"CL_cruise: {CL_cruise_2:.3f}")
    print(f"V_cruise: {V_cruise_2:.2f} m/s")
    print(f"n_turning: {n_turning_2:.2f}")
    print(f"v_turning: {v_turning_2:.2f} m/s")
    # np.degrees() returns a float, so we format the whole expression
    print(f"bank_angle: {np.degrees(bank_angle_rad2):.2f} degrees")
    print(f"turn_radius: {turn_radius_2:.2f} m")
    print(f"number_of_laps: {number_of_laps_2}") # Integer
    print(f"m2_score: {m2_score:.2f}")
    print()
    print("############ MISSION 3 ##############")
    print(f"CL_cruise: {CL_cruise_3:.3f}")
    print(f"V_cruise: {V_cruise_3:.2f} m/s")
    print(f"n_turning: {n_turning_3:.2f}")
    print(f"v_turning: {v_turning_3:.2f} m/s")
    print(f"bank_angle: {np.degrees(bank_angle_rad3):.2f} degrees")
    print(f"turn_radius: {turn_radius_3:.2f} m")
    print(f"number_of_laps: {number_of_laps_3}") # Integer
    print(f"m3_score: {m3_score:.2f}")
    
    # Final Objective (Minimizing negative score)
    final_score = (m3_score + m2_score)
    print()
    print("FINAL M2 + M3 SCORE: ", final_score)
    return - (m3_score + m2_score)

## Optimisation

In [21]:
#WS, pucks, PC_ratio, banner_length, banner_AR, m2_fly_time, m3_fly_time
search_space = [
    Real(0.9, 1.52, name="wing_span"),      # m
    Integer(1, 30, name="pucks"),
    Integer(3, 15, name="passenger_cargo_ratio"),
    Integer(10, 1000, name="banner_length"), #inch
    Integer(1, 5, name="banner_AR"),
    Integer(45, 270, name="m2_fly_time"), #s
    Integer(45, 270, name="m3_fly_time") #s
]

# Run Bayesian Optimization
result = gp_minimize(
    objective_total,
    search_space,
    n_calls=150,
    n_initial_points=10,
    random_state=42
)

# Best design
best_params = result.x
#WS, pucks, PC_ratio, banner_length, banner_AR, m2_fly_time, m3_fly_time
print("Best parameters found:", best_params)
objective_total(best_params)


############ General Plane Dimensions ###############
WS: 1.14
Chord: 0.14
Aspect Ratio: 8.07
Motor Power: 450.0
CDO: 0.1545

############ MISSION INPUTS ###############
Mission 2: Ducks = 6, Pucks = 1, payload = 0.28 kg, Battery Cap = 38 Wh
Mission 3: Banner Length (Inches)= 249.0, Banner AR = 4.0, payload = 0.90 kg, Battery Cap = 46 Wh

############ MISSION 2 ##############
CL_cruise: 0.200
V_cruise: 28.50 m/s
n_turning: 4.39
v_turning: 26.72 m/s
bank_angle: 76.84 degrees
turn_radius: 17.01 m
number_of_laps: 4.0
m2_score: 1.11

############ MISSION 3 ##############
CL_cruise: 0.506
V_cruise: 21.04 m/s
n_turning: 3.18
v_turning: 26.72 m/s
bank_angle: 71.69 degrees
turn_radius: 24.08 m
number_of_laps: 3.0
m3_score: 2.72

FINAL M2 + M3 SCORE:  3.839154946861761

############ General Plane Dimensions ###############
WS: 1.14
Chord: 0.15
Aspect Ratio: 7.83
Motor Power: 468.3
CDO: 0.1545

############ MISSION INPUTS ###############
Mission 2: Ducks = 6, Pucks = 1, payload = 0.28 kg, Batte

np.float64(-4.070588416706626)